In [1]:
%matplotlib inline

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt

import ipywidgets
from IPython.display import display

In [2]:
datapath = '/g/data1/v45/pm2987/Transfer_maps/'
datapath2 = '/g/data1/v45/pm2987/SpecTransfers_034_133/'
datapath3 = '/g/data1/v45/pm2987/QGCM_avgs/'
var = 'PE1'

# Define functions used later in the code

In [3]:
# Define array used in colorbar limits
arr = np.concatenate((np.arange(1,10),np.arange(10,100,10),np.arange(100,1000,100)))

# Function to find exponent of a number
def fexp(number):
    return np.int(np.log10(number))-1

# Function to find mantissa of a number
def fman(number):
    return int(number*(10**(-1*fexp(number))))

# Function to plot a given variable with sliders for min and max frequencies, and vmax
def plot_section(var, section, w_step_min, w_step_max, vmax, pcontour, hcontour):
    plt.figure(figsize=(12,8))
    
    # Take average of transfer
    if w_step_min == w_step_max: # plot single snapshot if min and max equal
        var_avg = var[:,:,w_step_min]
    else:
        var_avg = np.sum(var[:,:,w_step_min:w_step_max+1],axis=2)/(w_step_max-w_step_min+1)
    
    # Plot 2d transfer
    plt.imshow(var_avg, vmin=-vmax, vmax=vmax, cmap='bwr')
    plt.gca().invert_yaxis()  
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    #plt.plot(898,898,'ro')

    # Overlay a contour
    if pcontour:
        CS = plt.contour(p_avg,20,colors='k',alpha=0.7)
        plt.clabel(CS, inline=1)
    if hcontour:
        CS = plt.contour(h_avg,20,colors='grey',alpha=0.8)
        plt.clabel(CS, inline=1)
    
    # Plot 1d freq-domain transfer
    plt.figure(figsize=(8,6))
    freq = (2./(100*365))*np.arange(0,np.floor(100*365.0/2)+1)
    plt.semilogx(freq,T,color='b')
    
    # Plot circles indicating averaging range
    plt.plot(freq[200*(section-1)+w_step_min],T[200*(section-1)+w_step_min],'mo')
    plt.plot(freq[200*(section-1)+w_step_max],T[200*(section-1)+w_step_max],'mo')
    
    # Plot shaded area between min and max
    plt.fill_betweenx(np.linspace(np.min(T),np.max(T),num=500), 
                      freq[200*(section-1)+w_step_min], freq[200*(section-1)+w_step_max], 
                      facecolor='magenta', alpha=0.5)
    
    plt.axhline(0,color='k',linestyle='dotted')
    plt.xlim(freq[1],freq[-1])
    plt.ylim(np.min(T),np.max(T))  
    plt.show()

# Import spatially averaged transfer and time-averaged p,h

In [4]:
T = np.load(datapath2+var+'_oc_034_133_2cellmask.npy')
print(T.shape)

# Import time-average pressure
p_avg = np.load(datapath3+'p_timeAvg_059_108.npy')
h_avg = np.load(datapath3+'h_timeAvg_059_108.npy')
print(p_avg.shape)

(18251,)
(961, 961)


# Import map1

In [5]:
data = Dataset(datapath+var+'_map1_034_133.nc')
var1 = data.variables[var]
print(var1.shape)

(957, 957, 200)


## Interactive plot of map1

In [7]:
# Define maximum value for plotting bounds
vmax = np.max(np.max(np.max(var1)))
values=[i*10**(fexp(vmax)-3) for i in arr]
print(vmax,fexp(vmax))

ipywidgets.interact_manual(plot_section, var=ipywidgets.fixed(var1), section=ipywidgets.fixed(1),
                                     w_step_min=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               layout=ipywidgets.Layout(width='80%')),
                                      w_step_max=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               value=199,
                                               layout=ipywidgets.Layout(width='80%')),
                                      vmax=ipywidgets.SelectionSlider(
                                               options=[("%g"%i,i) for i in values],
                                               value=values[9],
                                               layout=ipywidgets.Layout(width='60%')), 
                                      pcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Pressure contours (black)',
                                                disabled=False),
                                      hcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Interface contours (grey)',
                                                disabled=False))  

2.05063e-10 -10


interactive(children=(IntSlider(value=0, description='w_step_min', layout=Layout(width='80%'), max=199), IntSlider(value=199, description='w_step_max', layout=Layout(width='80%'), max=199), SelectionSlider(description='vmax', index=9, layout=Layout(width='60%'), options=(('1e-13', 1e-13), ('2e-13', 2.0000000000000001e-13), ('3e-13', 3.0000000000000003e-13), ('4e-13', 4.0000000000000001e-13), ('5e-13', 4.9999999999999999e-13), ('6e-13', 6.0000000000000007e-13), ('7e-13', 7.0000000000000005e-13), ('8e-13', 8.0000000000000002e-13), ('9e-13', 9e-13), ('1e-12', 9.9999999999999998e-13), ('2e-12', 2e-12), ('3e-12', 3.0000000000000001e-12), ('4e-12', 3.9999999999999999e-12), ('5e-12', 5.0000000000000005e-12), ('6e-12', 6.0000000000000003e-12), ('7e-12', 7.0000000000000001e-12), ('8e-12', 7.9999999999999998e-12), ('9e-12', 8.9999999999999996e-12), ('1e-11', 1.0000000000000001e-11), ('2e-11', 2.0000000000000002e-11), ('3e-11', 3e-11), ('4e-11', 4.0000000000000004e-11), ('5e-11', 5.0000000000000002e-11), ('6e-11', 6e-11), ('7e-11', 7.0000000000000004e-11), ('8e-11', 8.0000000000000008e-11), ('9e-11', 8.9999999999999999e-11)), value=9.9999999999999998e-13), Checkbox(value=False, description='Pressure contours (black)'), Checkbox(value=False, description='Interface contours (grey)'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_section>

# Import map2

In [19]:
data = Dataset(datapath+var+'_map2_034_133.nc')
var2 = data.variables[var]
print(var2.shape)

(957, 957, 200)


## Interactive plot of map2

In [20]:
# Define maximum value for plotting bounds
vmax = np.max(np.max(np.max(var2)))
values=[i*10**(fexp(vmax)-3) for i in arr]
print(vmax,fexp(vmax))

ipywidgets.interact_manual(plot_section, var=ipywidgets.fixed(var2), section=ipywidgets.fixed(2),
                                     w_step_min=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               layout=ipywidgets.Layout(width='80%')),
                                      w_step_max=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               value=199,
                                               layout=ipywidgets.Layout(width='80%')),
                                      vmax=ipywidgets.SelectionSlider(
                                               options=[("%g"%i,i) for i in values],
                                               value=values[9],
                                               layout=ipywidgets.Layout(width='60%')), 
                                      pcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Pressure contours (black)',
                                                disabled=False),
                                      hcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Interface contours (grey)',
                                                disabled=False))    

1.94076e-10 -10


interactive(children=(IntSlider(value=0, description='w_step_min', layout=Layout(width='80%'), max=199), IntSlider(value=199, description='w_step_max', layout=Layout(width='80%'), max=199), SelectionSlider(description='vmax', index=9, layout=Layout(width='60%'), options=(('1e-13', 1e-13), ('2e-13', 2.0000000000000001e-13), ('3e-13', 3.0000000000000003e-13), ('4e-13', 4.0000000000000001e-13), ('5e-13', 4.9999999999999999e-13), ('6e-13', 6.0000000000000007e-13), ('7e-13', 7.0000000000000005e-13), ('8e-13', 8.0000000000000002e-13), ('9e-13', 9e-13), ('1e-12', 9.9999999999999998e-13), ('2e-12', 2e-12), ('3e-12', 3.0000000000000001e-12), ('4e-12', 3.9999999999999999e-12), ('5e-12', 5.0000000000000005e-12), ('6e-12', 6.0000000000000003e-12), ('7e-12', 7.0000000000000001e-12), ('8e-12', 7.9999999999999998e-12), ('9e-12', 8.9999999999999996e-12), ('1e-11', 1.0000000000000001e-11), ('2e-11', 2.0000000000000002e-11), ('3e-11', 3e-11), ('4e-11', 4.0000000000000004e-11), ('5e-11', 5.0000000000000002e-11), ('6e-11', 6e-11), ('7e-11', 7.0000000000000004e-11), ('8e-11', 8.0000000000000008e-11), ('9e-11', 8.9999999999999999e-11)), value=9.9999999999999998e-13), Checkbox(value=False, description='Pressure contours (black)'), Checkbox(value=False, description='Interface contours (grey)'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_section>

# Import map3

In [ ]:
data = Dataset(datapath+var+'_map3_034_133.nc')
var3 = data.variables[var]
print(var3.shape)

## Interactive plot of map3

In [ ]:
# Define maximum value for plotting bounds
vmax = np.max(np.max(np.max(var3)))
values=[i*10**(fexp(vmax)-3) for i in arr]
print(vmax,fexp(vmax))

ipywidgets.interact_manual(plot_section, var=ipywidgets.fixed(var3), section=ipywidgets.fixed(3),
                                     w_step_min=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               layout=ipywidgets.Layout(width='80%')),
                                      w_step_max=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               value=199,
                                               layout=ipywidgets.Layout(width='80%')),
                                      vmax=ipywidgets.SelectionSlider(
                                               options=[("%g"%i,i) for i in values],
                                               value=values[9],
                                               layout=ipywidgets.Layout(width='60%')), 
                                      pcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Pressure contours (black)',
                                                disabled=False),
                                      hcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Interface contours (grey)',
                                                disabled=False))    

# Import map5

In [13]:
data = Dataset(datapath+var+'_map5_034_133.nc')
var5 = data.variables[var]
print(var5.shape)

(957, 957, 200)


## Interactive plot of map5

In [15]:
# Define maximum value for plotting bounds
vmax = np.max(np.max(np.max(var5)))
values=[i*10**(fexp(vmax)-3) for i in arr]
print(vmax,fexp(vmax))

ipywidgets.interact_manual(plot_section, var=ipywidgets.fixed(var5), section=ipywidgets.fixed(5),
                                     w_step_min=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               layout=ipywidgets.Layout(width='80%')),
                                      w_step_max=ipywidgets.IntSlider(
                                               min=0,
                                               max=199,
                                               step=1,
                                               value=199,
                                               layout=ipywidgets.Layout(width='80%')),
                                      vmax=ipywidgets.SelectionSlider(
                                               options=[("%g"%i,i) for i in values],
                                               value=values[9],
                                               layout=ipywidgets.Layout(width='60%')), 
                                      pcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Pressure contours (black)',
                                                disabled=False),
                                      hcontour=ipywidgets.Checkbox(
                                                value=False,
                                                description='Interface contours (grey)',
                                                disabled=False))  

8.98702e-11 -11


interactive(children=(IntSlider(value=0, description='w_step_min', layout=Layout(width='80%'), max=199), IntSlider(value=199, description='w_step_max', layout=Layout(width='80%'), max=199), SelectionSlider(description='vmax', layout=Layout(width='60%'), options=(('1e-14', 1e-14), ('2e-14', 2e-14), ('3e-14', 2.9999999999999998e-14), ('4e-14', 4e-14), ('5e-14', 5.0000000000000002e-14), ('6e-14', 5.9999999999999997e-14), ('7e-14', 7.0000000000000005e-14), ('8e-14', 8e-14), ('9e-14', 8.9999999999999995e-14), ('1e-13', 1e-13), ('2e-13', 2.0000000000000001e-13), ('3e-13', 2.9999999999999998e-13), ('4e-13', 4.0000000000000001e-13), ('5e-13', 4.9999999999999999e-13), ('6e-13', 5.9999999999999997e-13), ('7e-13', 7.0000000000000005e-13), ('8e-13', 8.0000000000000002e-13), ('9e-13', 9e-13), ('1e-12', 9.9999999999999998e-13), ('2e-12', 2e-12), ('3e-12', 3.0000000000000001e-12), ('4e-12', 3.9999999999999999e-12), ('5e-12', 4.9999999999999997e-12), ('6e-12', 6.0000000000000003e-12), ('7e-12', 7.0000000000000001e-12), ('8e-12', 7.9999999999999998e-12), ('9e-12', 8.9999999999999996e-12)), value=1e-14), Checkbox(value=False, description='Pressure contours (black)'), Checkbox(value=False, description='Interface contours (grey)'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_section>